In [2]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch
import torch.nn.functional as F
import math
import numpy as np
import random       
import math 
from ray import tune
from hyperopt import hp, fmin, tpe, Trials
from functools import *
from ray.tune.logger import *
import time
import os
import copy
import datetime
from ray.tune.schedulers.pb2_utils import normalize, optimize_acq, \
            select_length, UCB, standardize, TV_SquaredExp


/opt/homebrew/Caskroom/miniforge/base/envs/gpbthebo/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# COMMON CODE

In [3]:
EPOCH_SIZE = 32*32*32*32
TEST_SIZE = 256*32*32 #remove 1024

#This is a function that can be used by several NN model
def train(model, optimizer ,func ,train_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.train()
    #for (data, target) in train_loader:
    for batch_idx, (data, target) in enumerate(train_loader):
        # We set this just for the example to run quickly.
        if batch_idx * len(data) > EPOCH_SIZE:
           # print("hehe")
            return
        # We set this just for the example to run quickly.
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = func(output, target)
        loss.backward()
        optimizer.step()
        
#This is a function that can be used by several NN model (it only does accuracy ATM)
def test(model, func, data_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(data_loader):
            # We set this just for the example to run quickly.
            data, target = data.to(device), target.to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)

            total += target.size(0)
            correct += (predicted == target).sum().item()  
    return correct / total

In [4]:
# A random mnist from the internet to get a correct model to reason about

class train_mnist():
    def __init__(self,config):
        self.DEFAULT_PATH = "./tmp/data/"
        self.config = {
        "sigmoid_func": 1
      ,  "hidden_dim":43
      ,  "n_layer":2    }
        for key, value in config.items():
            self.config[key] = value
        config = self.config
        
        self.i = 0
        
       # mnist_transforms = transforms.Compose(
       #     [transforms.ToTensor(),
       #      transforms.Normalize((0.1307, ), (0.3081, ))])
        mnist_transforms = transforms.ToTensor()

        self.train_loader = DataLoader(
            datasets.MNIST(self.DEFAULT_PATH, train=True, download=True , transform=mnist_transforms),
            batch_size=64,
            shuffle=True)
       # self.test_loader = DataLoader(
       #     datasets.MNIST(self.DEFAULT_PATH, train=False, transform=mnist_transforms),
       #     batch_size=64,
       #     shuffle=True)

        
        test_valid_dataset = datasets.MNIST(self.DEFAULT_PATH, train=False, transform=mnist_transforms)
        valid_ratio = 0.5  
        nb_test = int((1.0 - valid_ratio) * len(test_valid_dataset))
        nb_valid =  int(valid_ratio * len(test_valid_dataset))
        test_dataset, val_dataset = torch.utils.data.dataset.random_split(test_valid_dataset, [nb_test, nb_valid])
        self.test_loader =  DataLoader(test_dataset,
            batch_size=64,
            shuffle=True)

        self.val_loader =  DataLoader(val_dataset,
            batch_size=64,
            shuffle=True)

        sigmoid_func_uniq = nn.Tanh()


        self.model = LeNet(192,int(round(config.get("hidden_dim",64))),10,
                    int( round(config.get("n_layer",1))),
                    config.get("droupout_prob",0.5) ,sigmoid_func_uniq)
        
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=config.get("lr", 0.01), 
                                     betas=((config.get("b1", 0.999),config.get("b2", 0.9999))),
                                     eps=config.get("eps", 1e-08), 
                                     weight_decay=config.get("weight_decay", 0), 
                                     amsgrad=True)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(device)
    
    def adapt(self, config):
        #print(self.optimizer)
        temp = copy.deepcopy(self)
        for key, value in config.items():
            temp.config[key] = value
        config = temp.config

        temp.model.adapt(config.get("droupout_prob", 0.5))
        temp.optimizer = torch.optim.Adam(temp.model.parameters(), lr=config.get("lr", 0.01), 
                                     betas=((config.get("b1", 0.999),config.get("b2", 0.9999))),
                                     eps=config.get("eps", 1e-08), 
                                     weight_decay=config.get("weight_decay", 0), 
                                     amsgrad=True)
        return temp
    
    
# All NN models should have a function train1 and test1 that calls the common train and test defined above.
# train1 and test1 is then used in the scheduler
    def train1(self):
        print("iteration: " + str(self.i) )
        self.i+=1
        train(self.model, self.optimizer, F.nll_loss, self.train_loader)

    def val1(self):
        return test(self.model, F.nll_loss, self.val_loader)
        
    def test1(self):
        return test(self.model, F.nll_loss, self.test_loader)

    def step(self):
        self.train1()
        return self.val1()

# __INCEPTION_SCORE_begin__
class LeNet(nn.Module):
    """
    LeNet for MNist classification, used for inception_score
    """

    def __init__(self,input_dim, hidden_dim, output_dim, n_layers,
            drop_prob, sigmoid ):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d(drop_prob)
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def adapt(self,drop_prob):
        self.conv2_drop = nn.Dropout2d(drop_prob)
        
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)


## RANDOM

In [5]:
class RandomOpt():
    def __init__(self, searchspace ):
        self.searchspace = searchspace

    def compute_Once(self,function, evals): 
        fmin(function, self.searchspace, algo=partial(tpe.rand.suggest), max_evals=evals, trials=Trials())

class RandomLogger(tune.logger.Logger):
    def __init__(self, config):
        self.config = config
        timestamp = datetime.time().strftime("%H_%M_%d_%m_%Y")
        directory = os.path.join(".temp/data", "MNIST", timestamp)
        os.makedirs(directory, exist_ok=True)
        filename = "randomopt" + "_" + "LeNet" + "_" + str(0) + ".csv"
        progress_file = os.path.join(directory, filename)
        self._continuing = os.path.exists(progress_file)
        self._file = open(progress_file, "a")
        self._csv_out = None
    def on_result(self, result):
        tmp = result.copy()
        result = flatten_dict(tmp, delimiter="/")
        if self._csv_out is None:
            self._csv_out = csv.DictWriter(self._file, result.keys())
            self._csv_out.writeheader()

            #if not self._continuing:
        self._csv_out.writerow(
            {k: v
             for k, v in result.items() if k in self._csv_out.fieldnames})
        self._file.flush()

In [6]:
#Normal Random Opt
def function(x, hyperlogger):
    model = train_mnist(x)
    for _ in range(2): # Iterations
        model.train1()
        loss = model.test1()
        test = model.val1()
        temp = dict(x)
        temp.update({'loss' : loss})
        temp.update({'test' : test})

        temp.update({'iteration' :  model.i})
        hyperlogger.on_result(result=temp)
    return -loss

In [7]:
evals = 1  # num_configurations

config= {
     "lr": hp.uniform("lr",0,.1),
     "droupout_prob": hp.uniform("droupout_prob",0,1),
     "weight_decay": hp.uniform("weight_decay",0,.1),
    "b1" : hp.uniform("b1",0.9, 1),
    "b2" : hp.uniform("b2",0.99, 1)
}

oracle = RandomOpt(config)
hyperlogger = RandomLogger(config)
start_time = time.time()

fmin_objective = partial(function, hyperlogger=hyperlogger)
oracle.compute_Once(fmin_objective, evals)
print("totalt time: " +  str(time.time() - start_time))

iteration: 0                                         
iteration: 1                                         
100%|██████████| 1/1 [00:19<00:00, 19.50s/trial, best loss: -0.1088]
totalt time: 19.50960683822632


## BayesOPT

In [8]:
class BayesOpt():
    def __init__(self, searchspace ):
        self.searchspace = searchspace

    def compute_Once(self,function, evals): 
        fmin(function, self.searchspace, algo=partial(tpe.suggest, n_startup_jobs=1), max_evals=evals, trials=Trials())
    

class BayesLogger(tune.logger.Logger):
    def _init(self):
        progress_file = os.path.join("./tmp/data", "bayesopt_v2.csv")
        self._continuing = os.path.exists(progress_file)
        self._file = open(progress_file, "a")
        self._csv_out = None
    def on_result(self, result):
        tmp = result.copy()
        result = flatten_dict(tmp, delimiter="/")
        if self._csv_out is None:
            self._csv_out = csv.DictWriter(self._file, result.keys())
            self._csv_out.writeheader()

            #if not self._continuing:
        self._csv_out.writerow(
            {k: v
             for k, v in result.items() if k in self._csv_out.fieldnames})
        self._file.flush()

In [9]:
config= {
     "lr": hp.uniform("lr",0,.1),
     "droupout_prob": hp.uniform("droupout_prob",0,1),
     "weight_decay": hp.uniform("weight_decay",0,.1),
    "b1" : hp.uniform("b1",0.9, 1),
    "b2" : hp.uniform("b2",0.99, 1)
    
}

hyperlogger = BayesLogger(config,"")

for _ in range(1):
  oracle = BayesOpt(config)
  start_time = time.time()
  fmin_objective = partial(function, hyperlogger=hyperlogger)
  oracle.compute_Once(fmin_objective, evals)
  print("totalt time: " +  str(time.time() - start_time))

iteration: 0                                         
iteration: 1                                         
100%|██████████| 1/1 [00:18<00:00, 18.86s/trial, best loss: -0.8582]
totalt time: 18.863043069839478


## Hyperband

In [10]:
from ray.tune.schedulers.hb_bohb import HyperBandForBOHB
from ray.tune.suggest.bohb import *
class TestLogger(tune.logger.Logger):
    def _init(self):
        progress_file = os.path.join("./tmp/data/", "BOHB_LeNet.csv")
        self._continuing = os.path.exists(progress_file)
        self._file = open(progress_file, "a")
        self._csv_out = None
    def on_result(self, result):
        tmp = result.copy()
        result = flatten_dict(tmp, delimiter="/")
        if self._csv_out is None:
            self._csv_out = csv.DictWriter(self._file, result.keys())
            self._csv_out.writeheader()
        self._csv_out.writerow(
            {k: v
             for k, v in result.items() if k in self._csv_out.fieldnames})
        self._file.flush()

In [10]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
from ray import tune
from ray.tune.schedulers import ASHAScheduler
from hyperopt import hp
from ray.tune.suggest.hyperopt import HyperOptSearch
#from ray.tune.suggest.skopt import SkOptSearch
from ray import tune
from ray.tune.suggest.bayesopt import BayesOptSearch
import time
import ray
from ray.tune.schedulers import AsyncHyperBandScheduler
import argparse
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest.nevergrad import NevergradSearch
import json
from ray.tune import Trainable
from ray.tune.schedulers.hb_bohb import HyperBandForBOHB
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest.dragonfly import DragonflySearch
from ray.tune.schedulers import AsyncHyperBandScheduler
import torch

import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn as nn

import torch.nn.functional as F
from ray import tune
from ray.tune.schedulers import ASHAScheduler 
from ray.tune.schedulers import MedianStoppingRule

from hyperopt import hp
from ray.tune.suggest.hyperopt import HyperOptSearch
#from ray.tune.suggest.skopt import SkOptSearch
from ray import tune
from ray.tune.suggest.bayesopt import BayesOptSearch
import time
import ray
from ray.tune.schedulers import AsyncHyperBandScheduler
import argparse
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest.nevergrad import NevergradSearch
import json
import os
from ray.tune import Trainable
from ray.tune.schedulers.hb_bohb import HyperBandForBOHB
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.schedulers import AsyncHyperBandScheduler
#from ray.tune.suggest.dragonfly import DragonflySearch
from ray.tune.schedulers import AsyncHyperBandScheduler
import torch
#import adabelief_pytorch
global_checkpoint_period=np.inf
from ray.tune.schedulers.pb2 import PB2

class train_mnist_pb2(tune.Trainable):
    def setup(self, config):
        self.DEFAULT_PATH = "./tmp/data"
        self.config = {
        "sigmoid_func": 1
      ,  "hidden_dim":43
      ,  "n_layer":2    }
        for key, value in config.items():
            self.config[key] = value
        config = self.config
        
        self.i = 0
        
        #mnist_transforms = transforms.Compose(
        #    [transforms.ToTensor(),
        #     transforms.Normalize((0.1307, ), (0.3081, ))])
        mnist_transforms = transforms.ToTensor()

        self.train_loader = DataLoader(
            datasets.MNIST(self.DEFAULT_PATH, train=True, download=True, transform=mnist_transforms),
            batch_size=64,
            shuffle=True)
       # self.test_loader = DataLoader(
       #     datasets.MNIST("~/data", train=False, transform=mnist_transforms),
       #     batch_size=64,
       #     shuffle=True)

        
        test_valid_dataset = datasets.MNIST(self.DEFAULT_PATH, train=False, transform=mnist_transforms)
        valid_ratio = 0.5  
        nb_test = int((1.0 - valid_ratio) * len(test_valid_dataset))
        nb_valid =  int(valid_ratio * len(test_valid_dataset))
        test_dataset, val_dataset = torch.utils.data.dataset.random_split(test_valid_dataset, [nb_test, nb_valid])
        self.test_loader =  DataLoader(test_dataset,
            batch_size=64,
            shuffle=True)

        self.val_loader =  DataLoader(val_dataset,
            batch_size=64,
            shuffle=True)

        sigmoid_func_uniq = nn.Tanh()


        self.model = LeNet(192,int(round(config.get("hidden_dim",64))),10,
                    int( round(config.get("n_layer",1))),
                    config.get("droupout_prob",0.5) ,sigmoid_func_uniq)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(device)
        if config.get("b1", 0.999)>=1:
          temp = 1 - 1e-10
        else:
          temp = config.get("b1", 0.999)
                
        if config.get("b2", 0.999)>=1:
          temp1 = 1 - 1e-10
        else:
          temp1 = config.get("b2", 0.999)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=config.get("lr", 0.01), 
                                     betas=((temp,temp1)),
                                     eps=config.get("eps", 1e-08), 
                                     weight_decay=config.get("weight_decay", 0), 
                                     amsgrad=True)

    

    
    def reset_config(self, config):
        if "lr" in config:
            for param_group in self.optimizer.param_groups:
                param_group["lr"] = config.get("lr", 0.01)
        if "b1" in config:
            for param_group in self.optimizer.param_groups:
                param_group["betas"] = (
                    (1-1e-10 if config.get("b1", 0.999)>=1 else config.get("b1", 0.999),(1-1e-10 if config.get("b2", 0.999)>=1 else config.get("b2", 0.999))))
        if "weight_decay" in config:
            for param_group in self.optimizer.param_groups:
                param_group["weight_decay"] = config.get("weight_decay", 0) 
        
        self.model.adapt(config.get("droupout_prob", 0.5))

        self.config = config
        return True
    
# All NN models should have a function train1 and test1 that calls the common train and test defined above.
# train1 and test1 is then used in the scheduler
    def train1(self):
        print("iteration: " + str(self.i) )
        self.i+=1
        train(self.model, self.optimizer, F.nll_loss, self.train_loader)

    def val1(self):
        return test(self.model, F.nll_loss, self.val_loader)
        
    def test1(self):
        return test(self.model, F.nll_loss, self.test_loader)

    def step(self):
        self.train1()
        return {'loss' : self.test1(), 'test' : self.val1()}
    
    def save_checkpoint(self, checkpoint_dir):
        path = os.path.join(checkpoint_dir, "checkpoint")
        torch.save({
            "model": self.model.state_dict(),
            "optim": self.optimizer.state_dict(),

        }, path)

        return checkpoint_dir

    def load_checkpoint(self, checkpoint_dir):
        path = os.path.join(checkpoint_dir, "checkpoint")
        checkpoint = torch.load(path)
        self.model.load_state_dict(checkpoint["model"])
        self.optimizer.load_state_dict(checkpoint["optim"])

In [12]:
epsilon = 1e-10
config= {
  "lr":  tune.uniform(0+epsilon, .1) #tune.uniform(1e-4, 0.1 ),#,1e-4), #*10
  ,     "weight_decay":tune.uniform(0+epsilon, .1)#tune.uniform(1, 5)#,1e-4), #*10 et 0
  ,     "b1": tune.uniform(.9, 1-epsilon)#,1e-4), #*10 et 0
  ,    "b2": tune.uniform(.99, 1-epsilon)#,1e-4), #*10 et 0
  ,    "droupout_prob": tune.uniform(0+epsilon, 1-epsilon)#,1e-4), #*10 et 0
                  }

for _ in range(1):
    ray.shutdown()
    ray.init()
    start_time = time.time()
    algo = TuneBOHB(metric="loss", mode="max")
    bohb = HyperBandForBOHB(
        time_attr="training_iteration",
        metric="loss",
        mode="max",
        max_t=1)
    analysis = tune.run(train_mnist_pb2, config=config, scheduler=bohb, search_alg=algo,num_samples=1, callbacks=[CSVLoggerCallback],resources_per_trial={'cpu':8 ,'gpu': 1})

    print("time "+ str(time.time()- start_time))

## PB2

In [11]:
class TestLogger(tune.logger.Logger):
    def _init(self):
        progress_file = os.path.join("./temp/data", "PB2_2_6.csv")
        self._continuing = os.path.exists(progress_file)
        self._file = open(progress_file, "a")
        self._csv_out = None
    def on_result(self, result):
        tmp = result.copy()
        result = flatten_dict(tmp, delimiter="/")
        if self._csv_out is None:
            self._csv_out = csv.DictWriter(self._file, result.keys())
            self._csv_out.writeheader()
        self._csv_out.writerow(
            {k: v
             for k, v in result.items() if k in self._csv_out.fieldnames})
        self._file.flush()

In [ ]:


epsilon = 1e-10
algo = ConcurrencyLimiter(HyperOptSearch(metric="loss",
    mode="max"), max_concurrent=25)
scheduler = PB2(
    time_attr="training_iteration",
    perturbation_interval=2,
    hyperparam_bounds={
        # distribution for resampling
     "lr": [0+epsilon, .1] #tune.uniform(1e-4, 0.1 ),#,1e-4), #*10
,     "weight_decay":[0+epsilon, .1]#tune.uniform(1, 5)#,1e-4), #*10 et 0
,     "b1": [.9, 1-epsilon]#,1e-4), #*10 et 0
 ,    "b2": [.99, 1-epsilon] #,1e-4), #*10 et 0
 ,    "droupout_prob": [0+epsilon, 1-epsilon]#,1e-4), #*10 et 0
    }) 

imageSize = 32
cst=imageSize
total=cst*cst*3
def trial_name_id(trial):
    return f"{trial.trainable_name}_{trial.trial_id}"
from ray import tune


for _ in range(1):
  ray.shutdown()
  ray.init()
  start_time = time.time()
  analysis = tune.run(   
  train_mnist_pb2,
  scheduler=scheduler,
  reuse_actors=False,
  search_alg=algo,
  verbose=2,
  checkpoint_at_end=True,
  num_samples=28,
  # export_formats=[ExportFormat.MODEL],
              config= {
  "lr":  tune.uniform(0+epsilon, .1) #tune.uniform(1e-4, 0.1 ),#,1e-4), #*10
  ,     "weight_decay":tune.uniform(0+epsilon, .1)#tune.uniform(1, 5)#,1e-4), #*10 et 0
  ,     "b1": tune.uniform(.9, 1-epsilon)#,1e-4), #*10 et 0
  ,    "b2": tune.uniform(.99, 1-epsilon)#,1e-4), #*10 et 0
  ,    "droupout_prob": tune.uniform(0+epsilon, 1-epsilon)#,1e-4), #*10 et 0
          
      },      stop={
          "training_iteration": 10,
      },        metric="loss",
      mode="max"
,resources_per_trial={'cpu':8 ,'gpu': 1}
              ,     loggers=[TestLogger])
  print("time "+ str(time.time()- start_time))

## PBT

In [ ]:

import argparse

import ray
from ray import tune
from ray.tune.schedulers.pb2 import PB2
from ray.tune.schedulers import PopulationBasedTraining
from ray.tune.examples.pbt_function import pbt_function


for i in range(1,6):
    class TestLogger(tune.logger.Logger):
        def _init(self):
            progress_file = os.path.join(path, "PBT"+str(i)+".csv")
            self._continuing = os.path.exists(progress_file)
            self._file = open(progress_file, "a")
            self._csv_out = None
        def on_result(self, result):
            tmp = result.copy()
            result = flatten_dict(tmp, delimiter="/")
            if self._csv_out is None:
                self._csv_out = csv.DictWriter(self._file, result.keys())
                self._csv_out.writeheader()
            self._csv_out.writerow(
                {k: v
                 for k, v in result.items() if k in self._csv_out.fieldnames})
            self._file.flush()

    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--smoke-test", action="store_true", help="Finish quickly for testing")
    args, _ = parser.parse_known_args()
    ray.shutdown()
    ray.init()  # force pausing to happen for test

    epsilon = 1e-10
    from ray.tune.suggest.hyperopt import HyperOptSearch
    from ray.tune.suggest import ConcurrencyLimiter

    algo = HyperOptSearch(metric="loss",
        mode="max")
    algo = ConcurrencyLimiter(algo, max_concurrent=4)

    pbt = PopulationBasedTraining(
        perturbation_interval=1,
            time_attr="training_iteration",

        hyperparam_mutations={
            # hyperparameter bounds.
     "lr": [0+epsilon, .1] #tune.uniform(1e-4, 0.1 ),#,1e-4), #*10
,     "weight_decay":[0+epsilon, .1]#tune.uniform(1, 5)#,1e-4), #*10 et 0
,     "b1": [.9, 1-epsilon]#,1e-4), #*10 et 0
 ,    "b2": [.99, 1-epsilon] #,1e-4), #*10 et 0
 ,    "droupout_prob": [0+epsilon, 1-epsilon]#,1e-4), #*10 et 0
        })

    analysis = tune.run(
        train_mnist_pb2,
  checkpoint_at_end=True,
        scheduler=pbt,
        metric="loss",
        mode="max",
        search_alg = algo,
        verbose=0,
        stop={
            "training_iteration": 10,
        },
        num_samples=25,
  reuse_actors=True,
loggers=[TestLogger],
        
          config= {
  "lr":  tune.uniform(0+epsilon, .1) #tune.uniform(1e-4, 0.1 ),#,1e-4), #*10
  ,     "weight_decay":tune.uniform(0+epsilon, .1)#tune.uniform(1, 5)#,1e-4), #*10 et 0
  ,     "b1": tune.uniform(.9, 1-epsilon)#,1e-4), #*10 et 0
  ,    "b2": tune.uniform(.99, 1-epsilon)#,1e-4), #*10 et 0
  ,    "droupout_prob": tune.uniform(0+epsilon, 1-epsilon)#,1e-4), #*10 et 0
                      },resources_per_trial={'cpu':0 ,'gpu': 1})

    print("Best hyperparameters found were: ", analysis.best_config)

## GBPT

In [ ]:
# Oracle (Paul) TODO
path = "./temp/data"
class Oracle():
    def __init__(self, searchspace ):
        #self.hyperspace is the original (input) searchspace
        self.searchspace = searchspace

    def repeat_good(self,trials, iteration,function,configuration): #add space
        space = copy.deepcopy(configuration)
        for k,v in configuration.items():
            space[k] =  hp.uniform(k,-1e-10+v,v + 1e-10) 

        curr_eval = getattr(trials,'_ids')
        if curr_eval == set():
            curr_eval = 0
        else:
            curr_eval = max(curr_eval) +1
        space["itération"] =  hp.quniform("itération",-.5+iteration,.5+iteration, 1) 
        fmin(function, space, algo=partial(tpe.suggest, n_startup_jobs=1), max_evals=curr_eval
+1, trials=trials)
        
    def compute_once(self,trials, iteration,function): #add space

        space = copy.deepcopy(self.searchspace)
        curr_eval = getattr(trials,'_ids')
        if curr_eval == set():
            curr_eval = 0
        else:
            curr_eval = max(curr_eval) +1
        space["itération"] =  hp.quniform("itération",-.5+iteration,.5+iteration, 1) 
        fmin(function, space, algo=partial(tpe.suggest, n_startup_jobs=1), max_evals=curr_eval
+1, trials=trials)
        
        
    def compute_batch(self,trials, nb_eval, iteration,function): #add space

        space = copy.deepcopy(self.searchspace)
        curr_eval = getattr(trials,'_ids')
        if curr_eval == set():
            curr_eval = 0
        else:
            curr_eval = max(curr_eval) +1
            
        space["itération"] =  hp.quniform("itération",-.5+iteration,.5+iteration, 1) 
        fmin(function, space, algo=partial(tpe.suggest, n_startup_jobs=1), max_evals=curr_eval
+nb_eval, trials=trials)
    

In [ ]:
config= {
     "lr": hp.uniform("lr",0,.1)
    , "droupout_prob": hp.uniform("droupout_prob",0,1)
          ,   "weight_decay": hp.uniform("weight_decay",0,.1),
    "b1" : hp.uniform("b1",0.9, 1),
    "b2" : hp.uniform("b2",0.99, 1)
}
fsvnlogger = FSVNLogger(config,"")

    
CONFIGURATION = 25
ITERATIONS = 10

model = train_mnist
oracle = Oracle(config)


for _ in range(1):
  scheduler = Scheduler(
    model,
    ITERATIONS,
    CONFIGURATION,
    oracle) 

  start_time = time.time()
  scheduler.initialisation()     
  scheduler.loop()     
  print("totalt time: " +  str(time.time() - start_time))